In [1]:
!pip install fastf1

In [2]:
import os
if not os.path.exists('cache'):
    os.makedirs('cache')


In [3]:
# Import all the required libraries
import fastf1
import pandas as pd

fastf1.Cache.enable_cache('cache')  # Enable caching

# Get the 2024 F1 event schedule
schedule = fastf1.get_event_schedule(2024)

# Keep only the necessary columns
schedule = schedule[['RoundNumber', 'EventDate', 'EventName']]
schedule

,RoundNumber,EventDate,EventName
0,0,2024-02-23,Pre-Season Testing
1,1,2024-03-02,Bahrain Grand Prix
2,2,2024-03-09,Saudi Arabian Grand Prix
3,3,2024-03-24,Australian Grand Prix
4,4,2024-04-07,Japanese Grand Prix
5,5,2024-04-21,Chinese Grand Prix
6,6,2024-05-05,Miami Grand Prix
7,7,2024-05-19,Emilia Romagna Grand Prix
8,8,2024-05-26,Monaco Grand Prix
9,9,2024-06-09,Canadian Grand Prix


In [4]:
# Fetch all 2024 race results
results=[]
for i in range(1, len(schedule)): 
    session = fastf1.get_session(2024, i, 'R') 
    session.load()
        
    df = session.results
    df['Round'] = i    # Add race round number
    df['EventName'] = schedule.loc[schedule['RoundNumber'] == i, 'EventName'].values[0]  # Add event name
    
    results.append(df)

# Combine all race results into a single DataFrame
results_df = pd.concat(results, ignore_index=True)

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']
core           INFO 	Loading data for Saudi Arabian Grand Prix

In [5]:
print(results_df[results_df["EventName"] == "Las Vegas Grand Prix"])

    DriverNumber BroadcastName Abbreviation         DriverId         TeamName  \
419           63     G RUSSELL          RUS          russell         Mercedes   
420           44    L HAMILTON          HAM         hamilton         Mercedes   
421           55       C SAINZ          SAI            sainz          Ferrari   
422           16     C LECLERC          LEC          leclerc          Ferrari   
423            1  M VERSTAPPEN          VER   max_verstappen  Red Bull Racing   
424            4      L NORRIS          NOR           norris          McLaren   
425           81     O PIASTRI          PIA          piastri          McLaren   
426           27  N HULKENBERG          HUL       hulkenberg     Haas F1 Team   
427           22     Y TSUNODA          TSU          tsunoda               RB   
428           11       S PEREZ          PER            perez  Red Bull Racing   
429           14      F ALONSO          ALO           alonso     Aston Martin   
430           20   K MAGNUSS

In [6]:
schedule.shape

(25, 3)

In [7]:
print(schedule[schedule["EventName"] == "Abu Dhabi Grand Prix"]["RoundNumber"])

24    24
Name: RoundNumber, dtype: int64


In [8]:
# Fetch all 2025 race results till now

# Get event schedule for 2025
schedule = fastf1.get_event_schedule(2025)  # or fastf1.events.get_events(2025)

# The schedule has Session*DateUtc columns, e.g., Session5DateUtc for Race. :contentReference[oaicite:0]{index=0}
print(schedule.columns)

Index(['RoundNumber', 'Country', 'Location', 'OfficialEventName', 'EventDate',
       'EventName', 'EventFormat', 'Session1', 'Session1Date',
       'Session1DateUtc', 'Session2', 'Session2Date', 'Session2DateUtc',
       'Session3', 'Session3Date', 'Session3DateUtc', 'Session4',
       'Session4Date', 'Session4DateUtc', 'Session5', 'Session5Date',
       'Session5DateUtc', 'F1ApiSupport'],
      dtype='object')


In [9]:
from datetime import datetime, timezone
# Current UTC time
now_utc = datetime.now(timezone.utc).replace(tzinfo=None)

# Filter ONLY events whose race session has already occurred (Session5DateUtc)
past_events = schedule[schedule['Session5DateUtc'] <= pd.Timestamp(now_utc)]

print("Events completed up to now:")
print(past_events[['RoundNumber', 'EventName', 'Session5DateUtc']])

# ---- LOAD RACE RESULTS FOR THESE COMPLETED EVENTS ---- #

results2025 = []

for _,event in past_events.iterrows():
    rnd = int(event['RoundNumber'])

    print(f"Loading Round {rnd}: {event['EventName']} ...")
    
    session = fastf1.get_session(2025, rnd, 'R')   # 'R' = Race
    session.load()
    
    event_name = event['EventName']
    
    df = session.results
    df['Round'] = rnd
    df['EventName'] = event_name
    
    results2025.append(df)

results_2025_df = pd.concat(results2025, ignore_index=True)
print(results_2025_df)

core           INFO 	Loading data for Australian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


Events completed up to now:
    RoundNumber                  EventName     Session5DateUtc
1             1      Australian Grand Prix 2025-03-16 04:00:00
2             2         Chinese Grand Prix 2025-03-23 07:00:00
3             3        Japanese Grand Prix 2025-04-06 05:00:00
4             4         Bahrain Grand Prix 2025-04-13 15:00:00
5             5   Saudi Arabian Grand Prix 2025-04-20 17:00:00
6             6           Miami Grand Prix 2025-05-04 20:00:00
7             7  Emilia Romagna Grand Prix 2025-05-18 13:00:00
8             8          Monaco Grand Prix 2025-05-25 13:00:00
9             9         Spanish Grand Prix 2025-06-01 13:00:00
10           10        Canadian Grand Prix 2025-06-15 18:00:00
11           11        Austrian Grand Prix 2025-06-29 13:00:00
12           12         British Grand Prix 2025-07-06 14:00:00
13           13         Belgian Grand Prix 2025-07-27 13:00:00
14           14       Hungarian Grand Prix 2025-08-03 13:00:00
15           15           D

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '87'
core        WARNING 	Fixed incorrect tyre stint information for driver '30'
core        WARNING 	Fixed incorrect tyre stint information for driver '5'
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 4 completed the race distance 00:00.022000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['4', '1', '63', '12', '2

Loading Round 2: Chinese Grand Prix ...


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['81', '4', '63', '1', '31', '12', '23', '87', '18', '55', '6', '30', '7', '5', '27', '22', '14', '16', '44', '10']
core           INFO 	Loading data for Japanese Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


...

Loading Round 21: São Paulo Grand Prix ...


req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 4 completed the race distance 00:00.010000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['4', '12', '1', '63', '81', '87', '30', '6', '27', '10', '23', '31', '55', '14', '43', '18', '22', '44', '16', '5']
core           INFO 	Loading data for Las Vegas Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
Request for URL https://api.jolpi.ca/ergast/f1/2025/22/results.json failed; using cached response
Traceback (most recent call 

Loading Round 22: Las Vegas Grand Prix ...


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '63'
Request for URL https://api.jolpi.ca/ergast/f1/2025/22/laps/1.json failed; using cached response
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/site-packages/requests_cache/session.py", line 291, in _resend
    response.raise_for_status()
    ~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "/opt/anaconda3/lib/python3.13/site-packages/requests/models.py", line 1024, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 429 Client Error: Too Many Requests for url: https://api.jolpi.ca/ergast/f1/2025/22/laps/1.js

    DriverNumber BroadcastName Abbreviation        DriverId         TeamName  \
0              4      L NORRIS          NOR          norris          McLaren   
1              1  M VERSTAPPEN          VER  max_verstappen  Red Bull Racing   
2             63     G RUSSELL          RUS         russell         Mercedes   
3             12   A ANTONELLI          ANT       antonelli         Mercedes   
4             23       A ALBON          ALB           albon         Williams   
..           ...           ...          ...             ...              ...   
434           23       A ALBON          ALB           albon         Williams   
435            5   G BORTOLETO          BOR       bortoleto      Kick Sauber   
436           18      L STROLL          STR          stroll     Aston Martin   
437            4      L NORRIS          NOR          norris          McLaren   
438           81     O PIASTRI          PIA         piastri          McLaren   

    TeamColor        TeamId    FirstNam

In [10]:
results_2025_df.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,...,GridPosition,Q1,Q2,Q3,Time,Status,Points,Laps,Round,EventName
0,4,L NORRIS,NOR,norris,McLaren,FF8000,mclaren,Lando,Norris,Lando Norris,...,1.0,NaT,NaT,NaT,0 days 01:42:06.304000,Finished,25.0,57.0,1,Australian Grand Prix
1,1,M VERSTAPPEN,VER,max_verstappen,Red Bull Racing,3671C6,red_bull,Max,Verstappen,Max Verstappen,...,3.0,NaT,NaT,NaT,0 days 00:00:00.895000,Finished,18.0,57.0,1,Australian Grand Prix
2,63,G RUSSELL,RUS,russell,Mercedes,27F4D2,mercedes,George,Russell,George Russell,...,4.0,NaT,NaT,NaT,0 days 00:00:08.481000,Finished,15.0,57.0,1,Australian Grand Prix
3,12,A ANTONELLI,ANT,antonelli,Mercedes,27F4D2,mercedes,Andrea Kimi,Antonelli,Andrea Kimi Antonelli,...,16.0,NaT,NaT,NaT,0 days 00:00:10.135000,Finished,12.0,57.0,1,Australian Grand Prix
4,23,A ALBON,ALB,albon,Williams,64C4FF,williams,Alexander,Albon,Alexander Albon,...,6.0,NaT,NaT,NaT,0 days 00:00:12.773000,Finished,10.0,57.0,1,Australian Grand Prix


In [11]:

# Combine 2024 and 2025 race results into one DataFrame
results_df= pd.concat([results_df, results_2025_df], ignore_index=True,axis=0)

In [12]:
results_df.head(55)

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,...,GridPosition,Q1,Q2,Q3,Time,Status,Points,Laps,Round,EventName
0,1,M VERSTAPPEN,VER,max_verstappen,Red Bull Racing,3671c6,red_bull,Max,Verstappen,Max Verstappen,...,1.0,NaT,NaT,NaT,0 days 01:31:44.742000,Finished,26.0,57.0,1,Bahrain Grand Prix
1,11,S PEREZ,PER,perez,Red Bull Racing,3671c6,red_bull,Sergio,Perez,Sergio Perez,...,5.0,NaT,NaT,NaT,0 days 00:00:22.457000,Finished,18.0,57.0,1,Bahrain Grand Prix
2,55,C SAINZ,SAI,sainz,Ferrari,e8002d,ferrari,Carlos,Sainz,Carlos Sainz,...,4.0,NaT,NaT,NaT,0 days 00:00:25.110000,Finished,15.0,57.0,1,Bahrain Grand Prix
3,16,C LECLERC,LEC,leclerc,Ferrari,e8002d,ferrari,Charles,Leclerc,Charles Leclerc,...,2.0,NaT,NaT,NaT,0 days 00:00:39.669000,Finished,12.0,57.0,1,Bahrain Grand Prix
4,63,G RUSSELL,RUS,russell,Mercedes,27f4d2,mercedes,George,Russell,George Russell,...,3.0,NaT,NaT,NaT,0 days 00:00:46.788000,Finished,10.0,57.0,1,Bahrain Grand Prix
5,4,L NORRIS,NOR,norris,McLaren,ff8000,mclaren,Lando,Norris,Lando Norris,...,7.0,NaT,NaT,NaT,0 days 00:00:48.458000,Finished,8.0,57.0,1,Bahrain Grand Prix
6,44,L HAMILTON,HAM,hamilton,Mercedes,27f4d2,mercedes,Lewis,Hamilton,Lewis Hamilton,...,9.0,NaT,NaT,NaT,0 days 00:00:50.324000,Finished,6.0,57.0,1,Bahrain Grand Prix
7,81,O PIASTRI,PIA,piastri,McLaren,ff8000,mclaren,Oscar,Piastri,Oscar Piastri,...,8.0,NaT,NaT,NaT,0 days 00:00:56.082000,Finished,4.0,57.0,1,Bahrain Grand Prix
8,14,F ALONSO,ALO,alonso,Aston Martin,229971,aston_martin,Fernando,Alonso,Fernando Alonso,...,6.0,NaT,NaT,NaT,0 days 00:01:14.887000,Finished,2.0,57.0,1,Bahrain Grand Prix
9,18,L STROLL,STR,stroll,Aston Martin,229971,aston_martin,Lance,Stroll,Lance Stroll,...,12.0,NaT,NaT,NaT,0 days 00:01:33.216000,Finished,1.0,57.0,1,Bahrain Grand Prix


In [13]:
results_df.info()

<class 'fastf1.core.SessionResults'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   DriverNumber        918 non-null    object         
 1   BroadcastName       918 non-null    object         
 2   Abbreviation        918 non-null    object         
 3   DriverId            918 non-null    object         
 4   TeamName            918 non-null    object         
 5   TeamColor           918 non-null    object         
 6   TeamId              918 non-null    object         
 7   FirstName           918 non-null    object         
 8   LastName            918 non-null    object         
 9   FullName            918 non-null    object         
 10  HeadshotUrl         918 non-null    object         
 11  CountryCode         918 non-null    object         
 12  Position            918 non-null    float64        
 13  ClassifiedPosition  918 non-null    

In [14]:
results_2025_df.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,...,GridPosition,Q1,Q2,Q3,Time,Status,Points,Laps,Round,EventName
0,4,L NORRIS,NOR,norris,McLaren,FF8000,mclaren,Lando,Norris,Lando Norris,...,1.0,NaT,NaT,NaT,0 days 01:42:06.304000,Finished,25.0,57.0,1,Australian Grand Prix
1,1,M VERSTAPPEN,VER,max_verstappen,Red Bull Racing,3671C6,red_bull,Max,Verstappen,Max Verstappen,...,3.0,NaT,NaT,NaT,0 days 00:00:00.895000,Finished,18.0,57.0,1,Australian Grand Prix
2,63,G RUSSELL,RUS,russell,Mercedes,27F4D2,mercedes,George,Russell,George Russell,...,4.0,NaT,NaT,NaT,0 days 00:00:08.481000,Finished,15.0,57.0,1,Australian Grand Prix
3,12,A ANTONELLI,ANT,antonelli,Mercedes,27F4D2,mercedes,Andrea Kimi,Antonelli,Andrea Kimi Antonelli,...,16.0,NaT,NaT,NaT,0 days 00:00:10.135000,Finished,12.0,57.0,1,Australian Grand Prix
4,23,A ALBON,ALB,albon,Williams,64C4FF,williams,Alexander,Albon,Alexander Albon,...,6.0,NaT,NaT,NaT,0 days 00:00:12.773000,Finished,10.0,57.0,1,Australian Grand Prix


In [15]:
old_drivers = set(results_df['Abbreviation'].unique())   


new_drivers = set(results_2025_df['Abbreviation'].unique()) 
# Find drivers to ADD (exist in 2025 but NOT in 2024)
drivers_to_add = new_drivers - old_drivers
print("Drivers to add:", drivers_to_add)
# Find drivers to REMOVE (exist in 2024 but NOT in 2025)
drivers_to_remove = old_drivers - new_drivers
print("Drivers to remove:", drivers_to_remove)

# Remove old drivers who are not in 2025
results_df = results_df[~results_df['Abbreviation'].isin(drivers_to_remove)]

# Get data for new drivers from results2025_df
new_drivers_data = results_2025_df[results_2025_df['Abbreviation'].isin(drivers_to_add)]

# Add the new drivers to results_df
results_df = pd.concat([results_df, new_drivers_data], ignore_index=True)

results_df

Drivers to add: set()
Drivers to remove: {'SAR', 'MAG', 'PER', 'ZHO', 'BOT', 'RIC'}


,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,...,GridPosition,Q1,Q2,Q3,Time,Status,Points,Laps,Round,EventName
0,1,M VERSTAPPEN,VER,max_verstappen,Red Bull Racing,3671c6,red_bull,Max,Verstappen,Max Verstappen,...,1.0,NaT,NaT,NaT,0 days 01:31:44.742000,Finished,26.0,57.0,1,Bahrain Grand Prix
1,55,C SAINZ,SAI,sainz,Ferrari,e8002d,ferrari,Carlos,Sainz,Carlos Sainz,...,4.0,NaT,NaT,NaT,0 days 00:00:25.110000,Finished,15.0,57.0,1,Bahrain Grand Prix
2,16,C LECLERC,LEC,leclerc,Ferrari,e8002d,ferrari,Charles,Leclerc,Charles Leclerc,...,2.0,NaT,NaT,NaT,0 days 00:00:39.669000,Finished,12.0,57.0,1,Bahrain Grand Prix
3,63,G RUSSELL,RUS,russell,Mercedes,27f4d2,mercedes,George,Russell,George Russell,...,3.0,NaT,NaT,NaT,0 days 00:00:46.788000,Finished,10.0,57.0,1,Bahrain Grand Prix
4,4,L NORRIS,NOR,norris,McLaren,ff8000,mclaren,Lando,Norris,Lando Norris,...,7.0,NaT,NaT,NaT,0 days 00:00:48.458000,Finished,8.0,57.0,1,Bahrain Grand Prix
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,23,A ALBON,ALB,albon,Williams,1868DB,williams,Alexander,Albon,Alexander Albon,...,16.0,NaT,NaT,NaT,NaT,Retired,0.0,35.0,22,Las Vegas Grand Prix
788,5,G BORTOLETO,BOR,bortoleto,Kick Sauber,01C00E,sauber,Gabriel,Bortoleto,Gabriel Bortoleto,...,18.0,NaT,NaT,NaT,NaT,Retired,0.0,2.0,22,Las Vegas Grand Prix
789,18,L STROLL,STR,stroll,Aston Martin,229971,aston_martin,Lance,Stroll,Lance Stroll,...,12.0,NaT,NaT,NaT,NaT,Retired,0.0,0.0,22,Las Vegas Grand Prix
790,4,L NORRIS,NOR,norris,McLaren,F47600,mclaren,Lando,Norris,Lando Norris,...,1.0,NaT,NaT,NaT,NaT,Disqualified,0.0,50.0,22,Las Vegas Grand Prix


In [16]:
os.makedirs('model/DATA', exist_ok=True)
results_df.to_csv('model/DATA/f1_results_2024_2025.csv', index=False)

### Generating report for Driver's driving Experience

In [17]:
### Generating report for Driver's driving Experience
import pandas as pd
results_df = pd.read_csv('model/DATA/f1_results_2024_2025.csv')

results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792 entries, 0 to 791
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   DriverNumber        792 non-null    int64  
 1   BroadcastName       792 non-null    object 
 2   Abbreviation        792 non-null    object 
 3   DriverId            792 non-null    object 
 4   TeamName            792 non-null    object 
 5   TeamColor           792 non-null    object 
 6   TeamId              792 non-null    object 
 7   FirstName           792 non-null    object 
 8   LastName            792 non-null    object 
 9   FullName            792 non-null    object 
 10  HeadshotUrl         757 non-null    object 
 11  CountryCode         353 non-null    object 
 12  Position            792 non-null    float64
 13  ClassifiedPosition  792 non-null    object 
 14  GridPosition        792 non-null    float64
 15  Q1                  0 non-null      float64
 16  Q2      

In [18]:
driver_name = results_2025_df['FullName'].unique()
abbreviations = results_2025_df['Abbreviation'].unique()
TeamId = results_2025_df['TeamId'].unique()

print(len(driver_name))
print(len(abbreviations))
print(len(TeamId))

22
21
10


In [19]:

def compute_driver_experience(start_year=2018, end_year=2024):
    """
    Computes a single driver experience metric (total race starts)
    between start_year and end_year inclusive.
    
    Returns:
        DataFrame with columns:
            DriverName
            DriverExperience
    """

    # fastf1.Cache.enable_cache("cache")

    experience_records = []

    for year in range(start_year, end_year + 1):
        # Load schedule
        try:
            schedule = fastf1.get_event_schedule(year, include_testing=False)
        except:
            continue

        # Loop events
        for _, event in schedule.iterrows():
            try:
                session = fastf1.get_session(year, event["RoundNumber"], "R")
                session.load()
            except:
                continue

            # Race results must exist
            if session.results is None:
                continue

            # Count each driver appearing in race results
            for _, r in session.results.iterrows():
                if r.FullName in driver_name:
                    experience_records.append({
                        "FullName": r.FullName,
                        "Status": r.Status
                    })

    df = pd.DataFrame(experience_records)

    # A "RaceStart" = driver appears and did NOT have DNS
    df = df[df["Status"] != "Did not start"]

    # Count starts per driver
    experience = (
        df.groupby("FullName")
        .size()
        .reset_index(name="DriverExperience")
    )

    return experience

In [ ]:
exp = compute_driver_experience(2020, 2024)

print(exp)

core           INFO 	Loading data for Austrian Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Driver  8: Lap timing integrity check failed for 1 lap(s)
Request for URL https://api.jolpi.ca/ergast/f1/2020/1/laps/1.json failed; using cached response
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/site-packages/requests_cache/session.py", line 291, in _resend
    response.raise_for_status()
    ~~~~~~~~~~~~~~~~~~~~~~~~~^^
  File "/opt/anaconda3/lib/python3.13/site-packages/requests/models.py", line 1024, in raise

In [ ]:
drivers_experience = (
    results_2025_df[["Abbreviation", "FullName", "TeamId"]]
    .drop_duplicates(subset="Abbreviation", keep="first")
    .reset_index(drop=True)
)

drivers_experience = drivers_experience.merge(
    exp, on="FullName", how="left"
)

In [ ]:
drivers_experience.head(22)

,Abbreviation,FullName,TeamId,DriverExperience
0,NOR,Lando Norris,mclaren,51.0
1,VER,Max Verstappen,red_bull,51.0
2,RUS,George Russell,mercedes,51.0
3,ANT,Andrea Kimi Antonelli,mercedes,NaN
4,ALB,Alexander Albon,williams,31.0
5,STR,Lance Stroll,aston_martin,49.0
6,HUL,Nico Hulkenberg,sauber,17.0
7,LEC,Charles Leclerc,ferrari,51.0
8,PIA,Oscar Piastri,mclaren,15.0
9,HAM,Lewis Hamilton,ferrari,50.0


In [ ]:
# manual data corrections

drivers_list = drivers_experience.to_dict(orient='records')
print(drivers_list)

[{'Abbreviation': 'NOR', 'FullName': 'Lando Norris', 'TeamId': 'mclaren', 'DriverExperience': 51.0}, {'Abbreviation': 'VER', 'FullName': 'Max Verstappen', 'TeamId': 'red_bull', 'DriverExperience': 51.0}, {'Abbreviation': 'RUS', 'FullName': 'George Russell', 'TeamId': 'mercedes', 'DriverExperience': 51.0}, {'Abbreviation': 'ANT', 'FullName': 'Andrea Kimi Antonelli', 'TeamId': 'mercedes', 'DriverExperience': nan}, {'Abbreviation': 'ALB', 'FullName': 'Alexander Albon', 'TeamId': 'williams', 'DriverExperience': 31.0}, {'Abbreviation': 'STR', 'FullName': 'Lance Stroll', 'TeamId': 'aston_martin', 'DriverExperience': 49.0}, {'Abbreviation': 'HUL', 'FullName': 'Nico Hulkenberg', 'TeamId': 'sauber', 'DriverExperience': 17.0}, {'Abbreviation': 'LEC', 'FullName': 'Charles Leclerc', 'TeamId': 'ferrari', 'DriverExperience': 51.0}, {'Abbreviation': 'PIA', 'FullName': 'Oscar Piastri', 'TeamId': 'mclaren', 'DriverExperience': 15.0}, {'Abbreviation': 'HAM', 'FullName': 'Lewis Hamilton', 'TeamId': 'ferr

In [ ]:
# driver to remove
driver_to_remove ="Jack Doohan"

for driver in drivers_list:
    if driver['FullName'] == driver_to_remove:
        drivers_list.remove(driver)
        break
    
for driver in drivers_list:
    print(driver)

{'Abbreviation': 'NOR', 'FullName': 'Lando Norris', 'TeamId': 'mclaren', 'DriverExperience': 51.0}
{'Abbreviation': 'VER', 'FullName': 'Max Verstappen', 'TeamId': 'red_bull', 'DriverExperience': 51.0}
{'Abbreviation': 'RUS', 'FullName': 'George Russell', 'TeamId': 'mercedes', 'DriverExperience': 51.0}
{'Abbreviation': 'ANT', 'FullName': 'Andrea Kimi Antonelli', 'TeamId': 'mercedes', 'DriverExperience': nan}
{'Abbreviation': 'ALB', 'FullName': 'Alexander Albon', 'TeamId': 'williams', 'DriverExperience': 31.0}
{'Abbreviation': 'STR', 'FullName': 'Lance Stroll', 'TeamId': 'aston_martin', 'DriverExperience': 49.0}
{'Abbreviation': 'HUL', 'FullName': 'Nico Hulkenberg', 'TeamId': 'sauber', 'DriverExperience': 17.0}
{'Abbreviation': 'LEC', 'FullName': 'Charles Leclerc', 'TeamId': 'ferrari', 'DriverExperience': 51.0}
{'Abbreviation': 'PIA', 'FullName': 'Oscar Piastri', 'TeamId': 'mclaren', 'DriverExperience': 15.0}
{'Abbreviation': 'HAM', 'FullName': 'Lewis Hamilton', 'TeamId': 'ferrari', 'Dri

In [ ]:
driver_to_change = "Yuki Tsunoda"
for driver in drivers_list:
    if driver['FullName'] == driver_to_change:
        driver['TeamId'] = 'red_bull'
        break
    
driver_to_change = "Liam Lawson"
for driver in drivers_list:
    if driver['FullName'] == driver_to_change:
        driver['TeamId'] = 'rb'
        break

In [ ]:
for driver in drivers_list:
    print(driver)

{'Abbreviation': 'NOR', 'FullName': 'Lando Norris', 'TeamId': 'mclaren', 'DriverExperience': 51.0}
{'Abbreviation': 'VER', 'FullName': 'Max Verstappen', 'TeamId': 'red_bull', 'DriverExperience': 51.0}
{'Abbreviation': 'RUS', 'FullName': 'George Russell', 'TeamId': 'mercedes', 'DriverExperience': 51.0}
{'Abbreviation': 'ANT', 'FullName': 'Andrea Kimi Antonelli', 'TeamId': 'mercedes', 'DriverExperience': nan}
{'Abbreviation': 'ALB', 'FullName': 'Alexander Albon', 'TeamId': 'williams', 'DriverExperience': 31.0}
{'Abbreviation': 'STR', 'FullName': 'Lance Stroll', 'TeamId': 'aston_martin', 'DriverExperience': 49.0}
{'Abbreviation': 'HUL', 'FullName': 'Nico Hulkenberg', 'TeamId': 'sauber', 'DriverExperience': 17.0}
{'Abbreviation': 'LEC', 'FullName': 'Charles Leclerc', 'TeamId': 'ferrari', 'DriverExperience': 51.0}
{'Abbreviation': 'PIA', 'FullName': 'Oscar Piastri', 'TeamId': 'mclaren', 'DriverExperience': 15.0}
{'Abbreviation': 'HAM', 'FullName': 'Lewis Hamilton', 'TeamId': 'ferrari', 'Dri

In [ ]:
drivers_experience = pd.DataFrame(drivers_list)

drivers_experience.head(22)

,Abbreviation,FullName,TeamId,DriverExperience
0,NOR,Lando Norris,mclaren,51.0
1,VER,Max Verstappen,red_bull,51.0
2,RUS,George Russell,mercedes,51.0
3,ANT,Andrea Kimi Antonelli,mercedes,NaN
4,ALB,Alexander Albon,williams,31.0
5,STR,Lance Stroll,aston_martin,49.0
6,HUL,Nico Hulkenberg,sauber,17.0
7,LEC,Charles Leclerc,ferrari,51.0
8,PIA,Oscar Piastri,mclaren,15.0
9,HAM,Lewis Hamilton,ferrari,50.0


In [ ]:
drivers_experience["DriverExperience"].fillna(1, inplace=True)

drivers_experience.head(22)

/var/folders/fr/xqf7b3bn0fg_f_wgvw1pf2h80000gn/T/ipykernel_22081/3176493501.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  drivers_experience["DriverExperience"].fillna(1, inplace=True)


,Abbreviation,FullName,TeamId,DriverExperience
0,NOR,Lando Norris,mclaren,51.0
1,VER,Max Verstappen,red_bull,51.0
2,RUS,George Russell,mercedes,51.0
3,ANT,Andrea Kimi Antonelli,mercedes,1.0
4,ALB,Alexander Albon,williams,31.0
5,STR,Lance Stroll,aston_martin,49.0
6,HUL,Nico Hulkenberg,sauber,17.0
7,LEC,Charles Leclerc,ferrari,51.0
8,PIA,Oscar Piastri,mclaren,15.0
9,HAM,Lewis Hamilton,ferrari,50.0


In [ ]:
drivers_experience.to_excel('model/DATA/drivers_experience.xlsx', index=False)